In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import datetime
import math
import random

In [2]:
file_path = "./normal.csv"
df = pd.read_csv(file_path)
norvalue = df['value'].values

file_path = "./traindata.csv"
df = pd.read_csv(file_path)
convalue = df['value'].values

In [3]:
class Pid():
    def __init__(self, exp_val, con_val, kp, ki, kd):
        self.KP = kp
        self.KI = ki
        self.KD = kd
        self.exp_val = exp_val
        self.now_val = con_val
        self.sum_err = 0
        self.now_err = 0
        self.last_err = 0

    def cmd_pid(self):
        self.last_err = self.now_err
        self.now_err = self.exp_val - self.now_val
        self.sum_err += self.now_err
        # 这一块是严格按照公式来写的
        self.now_val = self.KP * (self.exp_val - self.now_val) \
                        + self.KI * self.sum_err + self.KD * (self.now_err - self.last_err)
        return self.now_val

In [5]:
def pid_soa(kpidi):
    lastvalue = []   #存储最后pid迭代的值
    for j in range(0,len(convalue)):   #一次控制2501个数据(即0.2s)，共用一个pid控制器
        my_Pid = Pid(norvalue[j], convalue[j], kpidi[0], kpidi[1], kpidi[2])   # nor正常  con异常  kp, ki, kd
        nowvalue = 0
        for i in range(1,3):  #每个数据迭代二次
            nowvalue = my_Pid.cmd_pid()  

        lastvalue.append(nowvalue)
                
    return np.linalg.norm(norvalue-lastvalue)

In [16]:
#种群初始化
Umax=0.9600   #最大隶属度值
Umin=0.0111   #最小隶属度值
wmax=0.9      #权重最大值
wmin=0.1
Dim=3         #维数
SwarmSize=35  #粒子群规模
MaxIter=100   #最大迭代次数
MinFit= 240   #最小适应值
Ub=np.array([0.5, 0.5, 0.5])    #PID参数的上限
Lb=np.array([-0.5,-0.5, -0.5])    #PID参数的下限
Indexfgbest=np.zeros((30,1))   #随机产生一个位置矩阵
Range=np.ones((SwarmSize,1))*(Ub-Lb)
Swarm=np.random.rand(SwarmSize,Dim)*Range+np.ones((SwarmSize,1))*Lb  #初始化粒子群 30x3矩阵
fSwarm=np.zeros((SwarmSize,1))   #计算粒子群适应值

for i in range(0,SwarmSize):
    fSwarm[i,:]=pid_soa(Swarm[i,:])    #粒子群的适应值，整个粒子群实参传给pidsoa，返回适应度值
    print(fSwarm[i],Swarm[i,:])

bestf=np.min(fSwarm)      #取最小适应值
bestin=np.argwhere(fSwarm==bestf)  #确定bestf在fSwarm矩阵的位置 ，得行和列的值
bestindex=bestin[0][0]      #取行索引
zbest=Swarm[bestindex,:]    #最佳Ki,Kd,Kp值
gbest=Swarm                 #所有Ki,Kd，Kp值
fgbest=fSwarm               #所有适应值
fzbest=bestf                #最佳适应值
# print(bestf,zbest,Swarm)
# print(fSwarm)

#迭代寻优
Di=0*np.random.rand(SwarmSize,Dim)
Buchang=0*np.random.rand(SwarmSize,Dim)
C=0*np.random.rand(SwarmSize,Dim)
Diego=0*np.random.rand(SwarmSize,Dim)
Dialt=0*np.random.rand(SwarmSize,Dim)
Dipro=0*np.random.rand(SwarmSize,Dim)
iter1=0
y_fitness=np.zeros((1,MaxIter))     #预先设置四个空矩阵
K_p=np.zeros((1,MaxIter))
K_i=np.zeros((1,MaxIter))
K_d=np.zeros((1,MaxIter))

[533.97142632] [-0.22793204  0.25492163  0.20401554]
[1148.60704966] [-0.04088688 -0.20597983 -0.16720431]
[592.50314391] [-0.13790725  0.1666481  -0.00473957]
[2838.81312992] [-0.32689867 -0.4296015  -0.37289538]
[1174.8195574] [-0.05858282 -0.2393667   0.0596707 ]
[1570.71317956] [-0.2530005  -0.27395974 -0.00165277]
[1902.31817111] [-0.04064392 -0.46486737 -0.18647045]
[1461.28183917] [ 0.2924771  -0.3737037   0.39298992]
[2479.57367555] [-0.49411766 -0.19863011 -0.4310475 ]
[619.20244428] [-0.33386832  0.258285    0.31577707]
[1302.8276978] [ 0.05997687 -0.30956764 -0.25488082]
[801.0342547] [-0.17139507  0.4845064  -0.20734957]
[840.1223045] [-0.36018415  0.38727158 -0.29230211]
[934.75819635] [ 0.12386452  0.3410422  -0.45901799]
[822.75284571] [ 0.48256212 -0.32150649 -0.32809323]
[470.53649375] [ 0.37568065  0.01719296 -0.16690011]
[923.51029321] [-0.39403977  0.18879722 -0.30162646]
[919.58570194] [0.32111655 0.04580414 0.23811498]
[737.17372602] [-0.08056069  0.03962455 -0.14

In [17]:
while (iter1<MaxIter) and (fzbest>MinFit):
# for j in range(0,MaxIter):
    for i in range(0,SwarmSize):
        w=wmax-iter1*(wmax-wmin)/MaxIter   #权重
        Diego[i,:]=np.sign(gbest[i,:]-Swarm[i,:])          #确定利己方向  个体最佳位置-目前位置
        Dialt[i,:]=np.sign(zbest-Swarm[i,:])               #确定利他方向  全局最佳位置-目前位置
        if pid_soa(gbest[i,:]) >= pid_soa(Swarm[i,:]):    #确定预动方向  目前位置-之前位置
            Dipro[i,:]=-Di[i,:]
        else:
            Dipro[i,:]=Di[i,:]
            
        Di[i,:]=np.sign(w*Dipro[i,:]+0.5*Diego[i,:]+0.5*Dialt[i,:])  #搜索方向  0.5可变动
        #确定经验梯度方向
        Orderfgbest=-np.sort(-fgbest,axis=0)    #个体最佳适应降序排列，默认升序 写两个负号变为降序
        Indexfgbest=np.argsort(-fgbest,axis=0)   #确定初始值的位置
        u=Umax-(SwarmSize-(Indexfgbest[i,0]+1))*(Umax-Umin)/(SwarmSize-1)  #搜索者个体i当前位置适应度的隶属度
        U=u+(1-u)*random.random()
        H=(MaxIter-iter1)/MaxIter  #迭代中权重的变化
        C[i,:]=H*abs(zbest-10*np.random.rand(1,3))  #确定高斯函数的参数
        T=math.sqrt(-math.log(U))
        Buchang[i,:]=C[i,:]*T    #确定搜索步长
        for n in range(0,Dim):
            if Buchang[i,n]>(3*np.max(C[i,:])):    #大于三倍参数，就等于这个参数
                Buchang[i,n]=3*np.max(C[i,:])
                
        #更新位置
        Swarm[i,:]=Swarm[i,:]+Di[i,:]*Buchang[i,:]   #对个体位置进行更新（即种群进化或最小优化问题候选解更新）
        for k in range(0,Dim):
            if Swarm[i,k]>Ub[k]:
                Swarm[i,k]=Ub[k]
            if Swarm[i,k]<Lb[k]:
                Swarm[i,k]=Lb[k]
        #适应值
        fSwarm[i,:]=pid_soa(Swarm[i,:])     
        #个体最优更新
        if fSwarm[i]<fgbest[i]:
            gbest[i,:]=Swarm[i,:]
            fgbest[i]=fSwarm[i]
            
        #群体最优更新
        if fSwarm[i]<fzbest:
            zbest=Swarm[i,:]
            print(Swarm[i,:],fSwarm[i,0])
            fzbest=fSwarm[i,0]
            print(zbest,fzbest)

#             K_p[0,iter1-1] = zbest[0]
#             K_i[0,iter1-1] = zbest[1]
#             K_d[0,iter1-1] = zbest[2]
        

    iter1 = iter1 + 1
#     print(zbest,fzbest)
#     print(zbest,fSwarm[i,0])
#     y_fitness[0,iter1-1] = fSwarm[i,0]   #单值出来
#     K_p[0,iter1-1] = zbest[0]
#     K_i[0,iter1-1] = zbest[1]
#     K_d[0,iter1-1] = zbest[2]

[-0.15309601  0.5         0.28857593] 443.6571740972198
[-0.15309601  0.5         0.28857593] 443.6571740972198
[ 0.13556766  0.48499588 -0.02386167] 248.2698934124932
[ 0.13556766  0.48499588 -0.02386167] 248.2698934124932
